In [154]:
import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import urllib
import tarfile
from nets import inception_v4 as inception
import tensorflow as tf

In [ ]:
def load_image(path):
    img = skimage.io.imread(path)
    img = img / 255
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy : yy + short_edge, xx : xx + short_edge]
    # resize to 224, 224
    resized_img = skimage.transform.resize(crop_img, (299, 299, 3))
    return resized_img.astype(np.float32)

def load_all_image(path, files):
    data = [load_image(path + "/" + file) for file in files]
    labels = [1 if "dog" in file else 0 for file in files]
    return np.array(data), np.array(labels)

def download_and_uncompress_tarball(tarball_url, dataset_dir):
    checkpoints_dir = '/tmp/checkpoints'
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)
    filename = tarball_url.split('/')[-1]
    filepath = os.path.join(dataset_dir, filename)
    if os.path.exists(filepath):
        print(filepath + ' exits')
    else:
        def _progress(count, block_size, total_size):
            sys.stdout.write('\r>> Downloading %s %.1f%%' % (filename, float(count * block_size) / float(total_size) * 100.0))
            sys.stdout.flush()
        filepath, _ = urllib.request.urlretrieve(tarball_url, filepath, _progress)
        statinfo = os.stat(filepath)
        print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
    tarfile.open(filepath, 'r:gz').extractall(dataset_dir)
    print('extractall', filepath)

def get_batchs(files, batch_size):
    length = len(files)
    total_batch = (length + batch_size - 1) // batch_size
    for i in range(total_batch):
        start = i * batch_size
        end = start + batch_size
        if end > length:
            end = length
        yield start, end, files[start: end]

# Preprocess Image : Extract Features

In [186]:
pretrain_data_url = 'http://download.tensorflow.org/models/inception_v4_2016_09_09.tar.gz'
checkpoints_dir = 'checkpoints_dir'
download_and_uncompress_tarball(pretrain_data_url, checkpoints_dir)

train_files = os.listdir('data/train')[0:2]
test_files = os.listdir('data/test')[0:2]
train_features = np.zeros((len(train_files), 1536))
train_labels = np.zeros(len(train_files))
test_features = np.zeros((len(test_files), 1536))
test_labels = np.zeros(len(test_files))

batch_size = 300

slim = tf.contrib.slim
pretrain_graph = tf.Graph()
with pretrain_graph.as_default():
    
    inputs = tf.placeholder(tf.float32, shape=(None, imge_size, imge_size, 3), name='inputs')
    
    with slim.arg_scope(inception.inception_v4_arg_scope()):
        logits, end_points = inception.inception_v4(inputs, is_training=False)
    
    probabilities = tf.nn.softmax(logits)
    PreLogitsFlatten = end_points['PreLogitsFlatten']
    
    init_fn = slim.assign_from_checkpoint_fn(
        os.path.join(checkpoints_dir, 'inception_v4.ckpt'),
        slim.get_model_variables('InceptionV4'))
    
    with tf.Session() as sess:
        init_fn(sess)
        print("load checkpoint success")
        
        for start, end, files in get_batchs(train_files, batch_size):
            data, labels = load_all_image('data/train', files)
            prob, logits = sess.run([probabilities, PreLogitsFlatten], feed_dict={inputs:data})
            print("process train batch ", start, end)
            
            train_features[start:end] = logits
            train_labels[start:end] = labels
            
        for start, end, files in get_batchs(test_files, batch_size):
            data, labels = load_all_image('data/test', files)
            prob, logits = sess.run([probabilities, PreLogitsFlatten], feed_dict={inputs:data})
            print("process test batch ", start, end)
            
            test_features[start:end] = logits
            test_labels[start:end] = labels
            
np.save("train_features.npy", train_features)
np.save("train_labels.npy", train_labels)
np.save("test_features.npy", test_features)
np.save("test_labels.npy", test_labels)
print("data saved")

checkpoints_dir/inception_v4_2016_09_09.tar.gz exits
extractall checkpoints_dir/inception_v4_2016_09_09.tar.gz
load checkpoint success
process train batch  0 2
process test batch  0 2
data saved


# Check Point

In [157]:
train_features = np.load("train_features.npy")
train_labels = np.load("train_labels.npy")
test_features = np.load("test_features.npy")
test_labels = np.load("test_labels.npy")

# Train on Image Features

In [187]:
def create_dog_cat_model(inputs, is_training=True, scope="dog_cat_model"):
    with tf.variable_scope(scope, 'dog_cat_model', [inputs]):
        end_points = {}
        
        with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):

            net = slim.fully_connected(inputs, 1024, scope='fc1')
            end_points['fc1'] = net

            net = slim.dropout(net, 0.8, is_training=is_training)

            net = slim.fully_connected(net, 1024, scope='fc2')
            end_points['fc2'] = net

            net = slim.fully_connected(net, 1, activation_fn=None, scope='logits')
            end_points['logits'] = net

            predictions = tf.nn.softmax(net, name='predictions')
            end_points['predictions'] = tf.nn.sigmoid(logits, name='predictions')
            return predictions, end_points

In [197]:
ckpt_dir = 'dog_cat_model'

dog_cat_graph = tf.Graph()
with dog_cat_graph.as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    inputs = tf.placeholder(tf.float64, shape=(None, 1536), name="inputs")
    labels = tf.placeholder(tf.float64, shape=(None), name="labels")

    predictions, end_points = create_dog_cat_model(inputs, is_training=True)
    loss = tf.losses.log_loss(labels, predictions)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(loss)
    init_op = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_op)
        
        loss_val,_ = sess.run([loss, train_op], feed_dict={inputs:train_features, labels:train_labels})
        print("Training Loss ", loss_val)

print("Training Finished")

Training Loss  16.1181
Training Finished


In [192]:
train_labels

array([ 0.,  0.])